In [ ]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "syntheticMRI2D"
FINAL_DATA_NAME = 'syntheticMRI-coronal-wavelet' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "cleaned"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data','syntheticMRI2D',"coronal")
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir

'c:\\Users\\riyap\\OneDrive\\Documents\\GitHub\\hierarchical-bayesian-model-validation\\raw-data\\syntheticMRI2D\\coronal'

In [9]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [10]:
file_list = [os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}"))
file_names[:5]

['.DS_Store-coronal-slice.npz',
 'sub-000001-coronal-slice.npz',
 'sub-000004-coronal-slice.npz',
 'sub-000009-coronal-slice.npz',
 'sub-000011-coronal-slice.npz']

In [11]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


full-cleaned
toy-cleaned



# Wavelet

In [12]:

batch_dir = os.path.join(ROOT_DIR, "raw-data", 'syntheticMRI2D',"coronal", f"full-{RAW_DATA_SUFFIX}")
orien_dict = {"horizontal":"H", "vertical":"V", "diagonal": "D"}

In [13]:
TRANSFORM = "wavelet"
channel = "gray"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x: x[~np.isnan(x)])
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
   
    pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    
min_group, max_group = 2, max(channel_wv['layer'])
for channel in ["horizontal", "vertical", "diagonal"]:
    orientation = orien_dict[channel]
    group_data_map = dict()
    group_data_map_size = dict()
    for group in np.arange(min_group, max_group + 1):
        data = np.array(channel_wv[(channel_wv['orientation'] == orientation) & (channel_wv['layer'] == group)]['data'].iloc[0])
        data = np.append(data, -data)
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
        group_data_map_size[group] = data.size
        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
                                
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

9 layers being used


100%|██████████| 5000/5000 [00:41<00:00, 119.56it/s]


,channel,layer,frequency,orientation,data
0,gray,1,0.110679,L1,[]
1,gray,2,0.124514,D,[]
2,gray,2,0.124514,H,[]
3,gray,2,0.124514,V,[]
4,gray,3,0.142301,D,"[-139.3518, 156.86722, 475.19525, -14.1328125,..."


In [14]:
del channel_wv, group_data_map, group_data_map_size